<a href="https://colab.research.google.com/github/GurmitSeera/Test_repo/blob/main/Window_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Window Function

Window functions applies aggregate and ranking functions over a particular window (set of rows). OVER clause is used with window functions to define that window. OVER clause does two things :



1.   Partitions rows into form set of rows. (PARTITION BY clause is used)
2.   Orders rows within those partitions into a particular order. (ORDER BY clause is used)



# Types of Window functions


1.   Aggregate Window Functions
2.   Ranking Window Functions

1.   Value Window Functions





In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS testdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE testdb;'
!pip install psycopg2

In [41]:
import pandas as pd
import psycopg2

In [42]:
connection_string = {'host':'localhost',
                     'dbname':'testdb',
                     'user':'postgres',
                     'password':'postgres',
                     'port':5432}
connection = psycopg2.connect(**connection_string)

# Aggregate Window Functions


1.   Sum()

1.   Max()
2.   Min()

1.   Avg()
2.   Count()





In [43]:
sql_query = """
Create table purchase (
  customer_id INT Primary Key,
  customer_name varchar(100),
  city varchar(100),
  amount double precision,
  purchasing_date date
)
"""
with connection.cursor() as cur:
  cur.execute(sql_query)

In [44]:
insert_query = """ insert into purchase(customer_id, customer_name, city, amount, purchasing_date)
values (1,'Manish','Sonipat',1000,'2021/01/05'),
       (2,'Raj','Noida',2500,'2021/01/08'),
       (3,'Shivani','Gurugram',1000,'2021/01/08'),
       (4,'Rohan','Gurugram',1200,'2021/01/08'),
       (5,'Sonu','Delhi',2800,'2021/01/09'),
       (6,'Manoj','Hisar',2500,'2021/01/09'),
       (7,'Deepak','Hisar',2400,'2021/01/09'),
       (8,'Rohit','Gurugram',1000,'2021/01/10'),
       (9,'Rohan','Gurugram',1200,'2021/01/10'),
       (10,'Raj','Noida',3500,'2021/01/11'),
       (11,'Yogesh','Hisar',3000,'2021/01/11'),
       (12,'Raj','Noida',3000,'2021/01/12'),
       (13,'Manish','Sonipat',1500,'2021/01/12'),
       (14,'Shivani','Gurugram',2000,'2021/01/13'),
       (15,'Rohan','Gurugram',2500,'2021/01/13'),
       (16,'Deepak','Hisar',2000,'2021/01/13'),
       (17,'Sonu','Delhi',1500,'2021/01/14'),
       (18,'Raj','Noida',2000,'2021/01/14'),
       (19,'Yogesh','Hisar',3800,'2021/01/14')
"""
with connection.cursor() as cur:
  cur.execute(insert_query)

In [ ]:
test_query = """ Select * from purchase;
"""
pd.read_sql_query(test_query,connection)

**# Problem 1. Sum()**

Calculate total amount spent everyday and date should be in descending order. Also display every customer_id, name and city.

In [46]:
sql_query1 = """ Select customer_id, customer_name, city, purchasing_date,
                  sum(amount) over (partition by purchasing_date order by purchasing_date desc) as total_amount
                  from purchase;
"""
pd.read_sql_query(sql_query1,connection)

,customer_id,customer_name,city,purchasing_date,total_amount
0,19,Yogesh,Hisar,2021-01-14,7300.0
1,18,Raj,Noida,2021-01-14,7300.0
2,17,Sonu,Delhi,2021-01-14,7300.0
3,14,Shivani,Gurugram,2021-01-13,6500.0
4,16,Deepak,Hisar,2021-01-13,6500.0
5,15,Rohan,Gurugram,2021-01-13,6500.0
6,13,Manish,Sonipat,2021-01-12,4500.0
7,12,Raj,Noida,2021-01-12,4500.0
8,10,Raj,Noida,2021-01-11,6500.0
9,11,Yogesh,Hisar,2021-01-11,6500.0
